Paquetes

In [ ]:
import requests
import configparser
import pandas as pd
import numpy as np
import json
import time as systime
import ast

Parametros de la cuenta (Demo)

In [ ]:
token = 'feda87f1f9a79663042fcd5ee82ab17c-22f342d828f6cfd14b6004a04fdb6283'
account = '101-011-11815575-001'
username = 'kgua0311'

Parametros de conexion a la API

In [ ]:
header = {"Content-Type": "application/json", "Authorization": "Bearer " + token}
pricing_url = 'https://api-fxpractice.oanda.com/v3/accounts/' + account + "/pricing"
order_url = 'https://api-fxpractice.oanda.com/v3/accounts/' + account + "/orders"
pricing_stream = 'https://stream-fxpractice.oanda.com/v3/accounts/' + account + "/pricing/stream"
instruments_url = "https://api-fxpractice.oanda.com/v3/accounts/" + account + "/instruments"

Instrumentos

In [ ]:
Tickers = ['USD_CNH', 'MBTC_USD', 'XAU_CAD', 'SGD_HKD', 'SPX500_USD', 'EUR_HUF', 'AUD_NZD', 'CAD_JPY',
           'EU50_EUR', 'USB10Y_USD', 'ZAR_JPY', 'EUR_AUD', 'DE30_EUR', 'XCU_USD', 'XAG_GBP', 'HKD_JPY',
           'XAG_JPY', 'XAG_USD', 'USD_NOK', 'CAD_SGD', 'USD_CZK', 'GBP_NZD', 'XAG_NZD', 'FR40_EUR', 
           'EUR_DKK', 'USD_CAD', 'AUD_JPY', 'USD_HUF', 'EUR_CZK', 'AUD_HKD', 'CHF_ZAR', 'SUGAR_USD',
           'EUR_HKD', 'AU200_AUD', 'NZD_USD', 'AUD_USD', 'SGD_JPY', 'AUD_CAD', 'WTICO_USD', 'DE10YB_EUR',
           'CORN_USD', 'AUD_CHF', 'CN50_USD', 'NL25_EUR', 'BTC_USD', 'TWIX_USD', 'EUR_CHF', 'USB02Y_USD',
           'SG30_SGD', 'EUR_NOK', 'AUD_SGD', 'USD_CHF', 'TRY_JPY', 'USD_THB', 'GBP_CHF', 'USD_PLN', 'GBP_AUD',
           'EUR_SGD', 'CHF_JPY', 'USD_INR', 'EUR_SEK', 'USD_SGD', 'XAU_NZD', 'XAU_GBP', 'USD_DKK', 'XPT_USD',
           'XAG_CHF', 'NZD_SGD', 'XAU_XAG', 'XAG_EUR', 'UK10YB_GBP', 'SOYBN_USD', 'GBP_ZAR', 'USD_HKD',
           'XAU_USD', 'GBP_PLN', 'USD_JPY', 'EUR_TRY', 'NAS100_USD', 'EUR_ZAR', 'XAU_HKD', 'NZD_HKD',
           'NZD_CHF', 'XAU_CHF', 'GBP_CAD', 'USD_SAR', 'XAU_JPY', 'NZD_CAD', 'EUR_JPY', 'NZD_JPY', 'EUR_NZD',
           'WHEAT_USD', 'XAU_AUD', 'US30_USD', 'XAU_EUR', 'UK100_GBP', 'EUR_GBP', 'USD_SEK', 'GBP_SGD',
           'US2000_USD', 'USB05Y_USD', 'USD_ZAR', 'NATGAS_USD', 'CHF_HKD', 'XPD_USD', 'GBP_HKD', 'EUR_PLN',
           'BCO_USD', 'XAG_AUD', 'JP225_USD', 'XAG_HKD', 'EUR_USD', 'XAG_SGD', 'XAU_SGD', 'IN50_USD',
           'USD_TRY', 'GBP_JPY', 'EUR_CAD', 'HK33_HKD', 'XAG_CAD', 'USD_MXN', 'GBP_USD', 'SGD_CHF', 
           'CAD_CHF', 'CAD_HKD', 'USB30Y_USD']

# Control para determinar si hacer stream de un solo activo o varios
multiple = True

if multiple:
    Tickers_stream = ','.join(Tickers)
else:
    Tickers_stream = 'USD_JPY' # Tomar uno en especifico

In [ ]:
def placeOrder(epic_id, order_type = None, units = None, price = None):
    """
    Function to place orders in OANDA.
    :param epic_id: Ticker of the instrument
    :param order_type: one of the following as strings
                - MARKET
                - LIMIT
    :param units: integer of the number of units to buy, negative if selling
    :param price: target price, only for LIMIT, STOP_LOSS, TAKE_PROFIT orders
    """

    if order_type is None:
        print('No type order given. Please specify the order type.')
        return None

    if order_type == "MARKET":
        params = {"order": {"timeInForce": "GTC",
                            "instrument": epic_id,
                            "units": str(units),
                            "type": order_type,
                            "positionFill": "DEFAULT"}}

    if order_type == "LIMIT":
        params = {"order": {"price": str(price),
                            "timeInForce": "GTC",
                            "instrument": epic_id,
                            "units": str(units),
                            "type": order_type,
                            "positionFill": "DEFAULT"}}

    params = json.dumps(params)
    session = requests.Session()
    req = requests.Request("POST", order_url, headers = header, data = params)
    pre = req.prepare()
    resp = session.send(pre)
    session.close()
    print(resp.reason)
    ins = resp.json()
    print(ins)

In [ ]:
url = "https://stream-fxpractice.oanda.com/v3/accounts/" + account + "/pricing/stream"
headers = {'Authorization':"Bearer " + token}
params = {"instruments":Tickers_stream}
s = requests.Session()
req = requests.Request("GET",url, headers = headers, params = params)
pre = req.prepare()
resp = s.send(pre, stream = True)
print(resp.reason)
for i in resp.iter_lines(1):
    j = i.decode()
    k = json.loads(j)
    if (k['type'] != 'HEARTBEAT'):
        print(k['instrument'])
        print(str(k['bids']) + ' / ' + str(k['asks']))
        print('\n')
    else:
        print(j)